# BCN open data SQL

In [5]:
import requests
import urllib.parse
import pandas as pd
from database import database
from pyproj import Transformer, CRS
import json
from tqdm.notebook import tqdm

url_base = 'https://opendata-ajuntament.barcelona.cat/data/api/action/datastore_search_sql?sql='
# Use generic "table" when writing the SQL query for clearer visualization

def convert_sql_query_to_bcnod_url (base, query, table_id):
    q = query.replace('table', table_id)
    q = urllib.parse.quote(q)
    return str(base + q)

ImportError: attempted relative import with no known parent package

In [2]:
table_id = '9bccce1b-0b9d-4cc6-94a7-459cb99450de'
sql_query = """
SELECT * from "table" 
"""

api_url = convert_sql_query_to_bcnod_url (url_base, sql_query, table_id)
response = requests.get(api_url).json()
df = pd.DataFrame(response['result']['records'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 441 entries, 0 to 440
Data columns (total 38 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   addresses_roadtype_name        441 non-null    object
 1   addresses_end_street_number    13 non-null     object
 2   institution_name               441 non-null    object
 3   values_attribute_name          441 non-null    object
 4   addresses_road_name            441 non-null    object
 5   values_category                441 non-null    object
 6   addresses_zip_code             441 non-null    object
 7   secondary_filters_id           441 non-null    object
 8   values_value                   441 non-null    object
 9   addresses_town                 441 non-null    object
 10  geo_epgs_4326_y                441 non-null    object
 11  geo_epgs_4326_x                441 non-null    object
 12  secondary_filters_name         441 non-null    object
 13  secon

# Data cleaning and formatting

In [3]:
#df = df.dropna()

# Convert columns values to numeric if possible, otherwise make sure they are stored as strings.
columns_to_num = df.columns.tolist()
for col in columns_to_num:
    try:
        df[col] = df[col].astype(str)
        df[col] = pd.to_numeric(df[col])
    except ValueError:
        df[col] = df[col].astype(str)
        pass
        #raise ('This column is not numeric')        
#df.info()

# Is it really necessary to delete columns?
columns_to_keep = ['geo_epgs_25831_x', 'geo_epgs_25831_y', 'name']
for c in df.columns:
    if (c not in columns_to_keep):
        df = df.drop(columns = c)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 441 entries, 0 to 440
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   geo_epgs_25831_x  441 non-null    float64
 1   name              441 non-null    object 
 2   geo_epgs_25831_y  441 non-null    float64
dtypes: float64(2), object(1)
memory usage: 10.5+ KB


# Insert in our database

In [4]:
f = open('config_file.json')
config = json.load(f)
table = "hotels"
my_db = database(config['host_name'], config['user_name'], config['password'], config['db'])

# Create table
my_db.execute_query(""" CREATE TABLE {table}(
id INT AUTO_INCREMENT,
NAME VARCHAR(100) NOT NULL,
EPSG_25831_X INT NOT NULL,
EPSG_25831_Y INT NOT NULL,
PRIMARY KEY (id)
)
""".format(table = table))


MySQL Server connection successful
MySQL Database connection test successful
Query successful


In [5]:
#Insert all data
col_names = ['EPSG_25831_X', 'NAME', 'EPSG_25831_Y']
columns = ", ".join(col_names)

query = 'INSERT INTO {table} ({columns}) VALUES '.format(columns = columns, table = table)
for row in tqdm(df.index):
    vals = ", ".join("'{s}'".format(s=s.replace("'", "''")) if isinstance(s, str) else str(s) for s in df.loc[row])
    query_line = "(" + vals + ") , "
    query = query + query_line
    
query = query[:-2] + ';'
#Insert all rows at once
my_db.execute_query(query)

  0%|          | 0/441 [00:00<?, ?it/s]

Query successful
